1. Generate data for project 1
2. Generate data for project 2

Input data:
For every planned path,
  for every step that the agent walks
    KnowledgeGrid:Local info
    no. of open cell,no. of blocked cells
  StartPosition:Global info

Output data:
  For every planned path,
    for every step that the agent walks
      which neighour the agent moves (left/right/up/down)

Loss function:
 ( Predicted neighbour that nn moves to - neighbour that agent moved to)^2

 Both are 4D vectors
 when agent moves left it is [1, 0, 0, 0]
 when agent moves right it is [0, 1, 0, 0]
 when agent moves up it is [0, 1, 0, 0]
 when agent moves down it is [0, 0, 0, 1]

First layer#
101*101 nodes
values of the nodes = 
{
  0 if cell was blocked
  1 if cell was open
  2 if this was start cell
}

1 hidden layer, which will be initialised with node values

3rd and final layer:
4 nodes 
Only 1 of the 4 nodes will have value = 1 (the one where the agent moved to given the input grid)
Rest are all 0

Every iteration in the executePlannedFunction is one data point
Input to executePlannedFunction is a planned path
Every iteration here means movement (l/r/u/d)

If the original is moving in such a way that it frequently encounters blocked cell
Then the nn should do that as well

Initially, the grid is all 0s all empty
First data point will be
All nodes will have value 0
One of the nodes will have value 2
If agent moved right
2nd node of Output layer will have value 1

Given 1000 examples of the above,

The NN would know that in this case, the agent chooses the neighbour to the right

How to use the NN to move our agent?

Once its trained,
you give it the same kind of input
Which means give it a newly generated grid, that was not part of the training data
If the output given is [1, 0, 0, 0]
Make the agent move left




knowledge.world.flatten
Converts the matrix into array that we pass to the first layer of nn

In [ ]:
import tensorflow as tf
import collections
from queue import PriorityQueue
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
from ast import literal_eval

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
GridData = pd.read_excel('/content/drive/My Drive/Intro to AI/AIFinalProject.xlsx')

In [ ]:
#GridData['Grid'] = GridData.Grid.apply(literal_eval)
#GridData.head()

In [ ]:
GridTest = GridData[0:1000]

In [ ]:
GridTest['Grid'] = GridTest.Grid.apply(literal_eval)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
GridTest['Movement'] = GridTest.Movement.apply(literal_eval)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Agent 1 -  OG Agent from Project 1

In [ ]:
MovementDict = {'Grid':[], 'Movement':[]}

In [ ]:
def Voyage(n,p):
  class Cell:
    # x = x-coordinate, y = y-coordinate in grid. Values from 0 to n-1
    def __init__(self, x, y,cost_from_start, total_cost, parent):
      self.x = x
      self.y = y
      self.cost_from_start = cost_from_start
      self.total_cost = total_cost
      self.parent = parent

    def __lt__(self, other):    # Added to add an object to the priority queue
        return self.total_cost < other.total_cost

  class Grid:
    def __init__(self, world, start, goal, path):
      self.iterations = 0
      self.world = world
      self.start = start
      self.goal = goal
      self.path = []
      self.restartValue = 0
      self.allCells = {}    # a dictionary to lookup a cell. Used when looking up its gScore

  def gridworld2(n,p):
    start = [0,0]; end = [n-1,n-1]
    gridworld = []

    for width in range(n):    # Loop creates n lists of n elements
      width = []
      for length in range(n):
        rand = np.random.random()
        if rand > p:
          width.append(0)
        else:
          width.append(1)
      gridworld.append(width)

    gridworld = np.array(gridworld)    # List of lists is turned into an array
    gridworld[start] = 0; gridworld[end] = 0
    return gridworld
  
  gridObj = Grid(gridworld2(n,p), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])
  knowledgeGrid = Grid(gridworld2(n,0), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])


  def isCellWithinGrid(x, y, n):
    if (x < 0) or (x > (n - 1)) or (y < 0) or (y > (n - 1)):
      return False

  def isCellOpen(cell, n, grid):
    if isCellWithinGrid(cell.x, cell.y, n) == False: # check if within gridworld bounds
      return False
    
    if grid[cell.x][cell.y] == 1:     # check if blocked
      return False
    
    return True

  def calculatefn(cell):
    goal = (n - 1, n - 1)
    estimate = calculateManhattanDistance(cell, goal)
    return estimate + cell.cost_from_start

  def calculateManhattanDistance(cell, goal):
    return abs(cell.x - goal[0]) + abs(cell.y - goal[1])

  # this is the planning phase
  def AStar(currentGrid, start):
    queue = PriorityQueue()
    gridObj.allCells = {}
    gridObj.allCells[(start.x, start.y)] = start
    queue.put((start.total_cost,start))    # first value in the tuple is priority(cost), second is the cell object
    goal = gridObj.goal

    while not queue.empty():   #while queue is not empty
      getter = queue.get()
      current = getter[1]

      if current.x == goal.x and current.y == goal.y:
        path = reconstructPath(current, start)
        return executePlannedPath(path) 
     
      rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))     # right neighbour
      downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0, (current.x, current.y))    # down neighbour
      leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))  # left neighbour
      topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0, (current.x, current.y))  # top neighbour
     
      neighbors = [rightCell, downCell, leftCell, topCell] # calculate the f(n) for each neighbour of current

      for i in neighbors:
        i.total_cost = calculatefn(i)         # calculate the f(n) for each neighbour of current
        isItOpen = isCellOpen(i, n, currentGrid)
        if isCellOpen(i,n, currentGrid):
          element = gridObj.allCells.get((i.x, i.y))
          if element is not None:
            ''' 
            If new path to neigbour costs less than previous path to it, then update cell.
            This cell has already been visited before, but we are just checking
            if the current path is less costlier than the one visited before
            '''
            if i.total_cost < gridObj.allCells[(i.x, i.y)].total_cost:
             gridObj.allCells[(i.x, i.y)] = i

          else:
           gridObj.allCells[(i.x, i.y)] = i      # this cell is being added first time
           queue.put((i.total_cost, i))
    
    # queue is empty but goal node has not been reached
    return False


  def reconstructPath(current, start):
    path = collections.deque()
    
    while True:
      if (current.x, current.y) == (start.x, start.y):
        break
      path.appendleft(current)
      current = gridObj.allCells[(current.parent[0], current.parent[1])]
    
    return path
  def directionArray(cell):
    if (cell.x-cell.parent[0], cell.y-cell.parent[1]) == (0,1):
      return [1,0,0,0] # Right
    if (cell.x-cell.parent[0], cell.y-cell.parent[1]) == (0,-1):
      return [0,1,0,0] # Left
    if (cell.x-cell.parent[0], cell.y-cell.parent[1]) == (-1,0):
      return [0,0,1,0] # Up
    if (cell.x-cell.parent[0], cell.y-cell.parent[1]) == (1,0):
      return [0,0,0,1] # Down

  def executePlannedPath(path):
    grid = gridObj.world
    goal = gridObj.goal
    isPathFeasible = True

    for index, current in enumerate(path):
      #print("Visiting ", current)
      gridObj.iterations += 1

      #print(current.x-current.parent[0], current.y-current.parent[1])
      MovementDict['Movement'].append(directionArray(current))
      og = knowledgeGrid.world[current.parent[0],current.parent[1]]
      knowledgeGrid.world[current.parent[0],current.parent[1]]=2
      #print(knowledgeGrid.world)
      MovementDict['Grid'].append(knowledgeGrid.world.flatten().tolist())
      knowledgeGrid.world[current.parent[0],current.parent[1]] = og
      if current.x == goal.x and current.y == goal.y:
        MovementDict['Movement'].append([0,0,0,0])
        og = knowledgeGrid.world[current.x,current.y]
        knowledgeGrid.world[current.x,current.y]=2
        #print(knowledgeGrid.world)
        MovementDict['Grid'].append(knowledgeGrid.world.flatten().tolist())
        knowledgeGrid.world[current.x,current.y] = og
        return gridObj.path

      if grid[current.x][current.y] == 1:
        knowledgeGrid.world[current.x][current.y] = 1
        isPathFeasible = False

        # this should be the restart point now
        setRestartPoint(gridObj.allCells[current.parent[0], current.parent[1]], 0)
        return None

      else:
        gridObj.path.append(current)

      # update the neighbours in the actual grid   
      # update the blockedNeighboursCount   
      if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
        knowledgeGrid[current.x + 1, current.y] = 1
      
      if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
        knowledgeGrid[current.x][current.y + 1] = 1
      
      if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x - 1][current.y] == 1:
        knowledgeGrid[current.x + 1][current.y] = 1
      
      if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x][current.y - 1] == 1:
        knowledgeGrid[current.x + 1][current.y] = 1
  
  def setRestartPoint(restartPoint, restartValue):
    for i in range(0, restartValue, 1):
      restartPoint = gridObj.allCells[(restartPoint.parent[0], restartPoint.parent[1])]

    gridObj.start = restartPoint
    return

  def restartController(current):
    # in a loop run the restart again and again
    # starting from the current cell's parent to further up
    while (current.x != 0 and current.y != 0):
      current = gridObj.allCells[(current.parent[0], current.parent[1])]
      gridObj.start = current

  def repeatedAStar():
    while True:
        if len(gridObj.path) > 0:
          previousStartPoint = gridObj.path[len(gridObj.path)-1]

          # if we are restarting from the same point where we restarted previously, then dont add this to the path again
          if gridObj.start.x != previousStartPoint.x and gridObj.start.y != previousStartPoint.y:
            gridObj.path.append(gridObj.start)

        path = AStar(knowledgeGrid.world, gridObj.start)
        if path is None:
          continue

        elif path == False:
          return False

        # final path to goal found
        else:
          gridObj.world[0][0] = 5
          gridObj.world[gridObj.start.x][gridObj.start.y] = 5
          
          for i in gridObj.path:
            gridObj.world[i.x][i.y] = 5

          gridObj.world[gridObj.goal.x][gridObj.goal.y] = 5
          print(gridObj.world)
          print("Total iterations = ", gridObj.iterations)
          return True

  def restartAtDifferentPoints():
    for i in range(0, (n*n - 1)):
      repeatedAStar()
    return

  return repeatedAStar()

Voyage(101, 0.3)

[[5 5 5 ... 0 0 0]
 [1 1 5 ... 0 1 1]
 [0 0 0 ... 1 0 1]
 ...
 [1 0 0 ... 1 0 0]
 [1 0 1 ... 0 0 1]
 [0 0 0 ... 5 5 5]]
Total iterations =  407


True

In [ ]:
#MovementFrame = pd.DataFrame(MovementDict)
#print(MovementFrame)
Grids = np.asarray(GridTest['Grid'])
Moves = np.asarray(GridTest['Movement'])
(x_train, y_train ) = (Grids, Moves)
print('Number of grids =', len(GridTest['Grid']))
print('x_train')
print('size of x_train', x_train.shape)
#print(x_train)

print('Number of movements =', len(GridTest['Movement']))
print('y_train')
print('size of y_train', y_train.shape)
#print(y_train)

Number of grids = 1000
x_train
size of x_train (1000,)
Number of movements = 1000
y_train
size of y_train (1000,)


In [ ]:
#MovementFrame['Grid']

In [ ]:
#GridData['Grid'][0]

  [0, 0, 0, 0] in the output corresponds to reaching goal

# Agent 3 -  OG Agent from Project 2

In [ ]:
MovementDict = {}

In [ ]:
def PartialSensing(n,p):
  class Cell:
    # x = x-coordinate, y = y-coordinate in grid. Values from 0 to n-1
    def __init__(self, x, y,cost_from_start, total_cost, parent):
      self.x = x
      self.y = y
      self.cost_from_start = cost_from_start
      self.total_cost = total_cost
      self.total_neighbours = 0
      self.total_blocked_neighbours = 0
      self.known_neighbours = 0
      self.known_blocked_neighbours = 0
      self.parent = parent
      self.confirmed = False
      self.visited = False

    def __lt__(self, other):    # Added to add an object to the priority queue
        return self.total_cost < other.total_cost

  class Grid:
    def __init__(self, world, start, goal, path):
      self.world = world
      self.start = start
      self.goal = goal
      self.path = []
      self.cellMap = {}    # a dictionary to lookup a cell.
    
  def gridworld2(n,p):
    start = [0,0]; end = [n-1,n-1]
    gridworld = []

    for width in range(n):    # Loop creates n lists of n elements
      width = []
      for length in range(n):
        rand = np.random.random()
        if rand > p:
          width.append(0)
        else:
          width.append(1)
      gridworld.append(width)

    gridworld = np.array(gridworld)    # List of lists is turned into an array
    gridworld[start] = 0; gridworld[end] = 0
    return gridworld
  
  gridObj = Grid(gridworld2(n,p), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])
  knowledgeGrid = Grid(gridworld2(n,0), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])

  def isCellWithinGrid(x, y, n):
    if (x < 0) or (x > (n - 1)) or (y < 0) or (y > (n - 1)):
      return False
    return True

  def isCellOpen(cell, n, grid):
    if isCellWithinGrid(cell.x, cell.y, n) == False: # check if within gridworld bounds
      return False
    
    if grid.world[cell.x, cell.y] == 1:     # check if blocked
      return False
    
    return True

  def calculatefn(cell):
    goal = (n - 1, n - 1)
    estimate = calculateManhattanDistance(cell, goal)
    return estimate + cell.cost_from_start

  def calculateManhattanDistance(cell, goal):
    return abs(cell.x - goal[0]) + abs(cell.y - goal[1])
    
  def getNeighbours(cell):
    #east, south, west, north, ne, se, sw, nw in that order
    x = [1, 0, -1, 0, 1,  1, -1, -1]
    y = [0, 1, 0, -1, -1, 1, 1, -1]

    neighbours = []
    for i in range(8):
      if isCellWithinGrid(cell.x + x[i], cell.y + y[i], n):
        if knowledgeGrid.cellMap.get((cell.x + x[i], cell.y + y[i])) is None:
          knowledgeGrid.cellMap[(cell.x + x[i], cell.y + y[i])] = Cell(cell.x + x[i], cell.y + y[i], 0, 0, (cell.x, cell.y))
        neighbours.append(knowledgeGrid.cellMap[(cell.x + x[i], cell.y + y[i])])

    return neighbours

  def senseNeighbours(cell):
    ECell = Cell(cell.x + 1, cell.y, cell.cost_from_start + 1, 0, (cell.x, cell.y))     # East neighbour
    SCell = Cell(cell.x, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))    # South neighbour
    WCell = Cell(cell.x - 1, cell.y, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # West neighbour
    NCell = Cell(cell.x, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North neighbour
    NECell = Cell(cell.x + 1, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))     # North East neighbour
    SECell = Cell(cell.x + 1, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))    # South East neighbour
    SWCell = Cell(cell.x - 1, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South West neighbour
    NWCell = Cell(cell.x - 1, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North West neighbour
    
    potential_neighbours = [ECell, SCell, WCell, NCell, NECell, SECell, SWCell, NWCell]
    neighbours = [i for i in potential_neighbours if isCellWithinGrid(i.x, i.y, n)]
    total = 0
    blocked = 0
    for i in neighbours:
      total += 1
      if gridObj.world[i.x, i.y] == 1:
        blocked += 1
    cell.total_neighbours = total
    cell.total_blocked_neighbours = blocked
    return cell

  def getKnownNeighbours(cell):
    #east, south, west, north, ne, se, sw, nw in that order
    x = [1, 0, -1, 0, 1,  1, -1, -1]
    y = [0, 1, 0, -1, -1, 1, 1, -1]

    neighbours = []
    for i in range(8):
      if isCellWithinGrid(cell.x + x[i], cell.y + y[i], n):
        neighbourCell = knowledgeGrid.cellMap.get((cell.x + x[i], cell.y + y[i]))
        if neighbourCell is not None and neighbourCell.confirmed:
          cell.known_neighbours += 1 
          if knowledgeGrid.world[cell.x + x[i], cell.y + y[i]] == 1:
            cell.known_blocked_neighbours += 1  
    return cell

  def updateKnownNeighbours(cell):
      knowledgeGrid.cellMap[(cell.x, cell.y)].known_neighbours += 1
      inferenceEngine(cell)

  def updateKnownBlockedNeighbours(cell):
      cell.known_blocked_neighbours += 1
      cell.known_neighbours += 1
      knowledgeGrid.cellMap[(cell.x, cell.y)] = cell
      inferenceEngine(cell)

  def inferenceEngine(cell):
    if cell.total_blocked_neighbours == cell.known_blocked_neighbours:
        markRemainingNeighboursAsFree(cell)

    # if totalFreeNeighbours = knownFreeNeighbours 
    if (cell.total_neighbours - cell.total_blocked_neighbours 
        == (cell.known_neighbours - cell.known_blocked_neighbours)):
        markRemainingNeighboursAsBlocked(cell)

    # Early Exit Function 
    if cell.total_blocked_neighbours == cell.total_neighbours - 1:
      markRemainingNeighboursAsBlocked(cell)

  def markRemainingNeighboursAsFree(cell):  
    neighbours = getNeighbours(cell)
    for i in neighbours:
      #this new freely marked cell must inform its neighbours which have been visited
      if i.confirmed == False:
        if gridObj.world[i.x, i.y] != 0:
          knowledgeGrid.world[i.x, i.y] = 0
        cell.known_neighbours += 1 
        knowledgeGrid.cellMap[(i.x, i.y)] = cell
        i.confirmed = True
        knowledgeGrid.cellMap[(i.x, i.y)] = i
        #inform the neighbours
        informNeigboursAboutNewFreeCell(i)
  
  def markRemainingNeighboursAsBlocked(cell):
    neighbours = getNeighbours(cell)
    for i in neighbours:
      #this new blocked cell must inform its neighbours which have been visited
      if i.confirmed == False:
        cell.known_blocked_neighbours += 1 
        cell.known_neighbours += 1 
        knowledgeGrid.cellMap[(i.x, i.y)] = cell
        if gridObj.world[i.x, i.y] != 1:
          knowledgeGrid.world[i.x, i.y] = 1
        i.confirmed  = True
        knowledgeGrid.cellMap[(i.x, i.y)] = i
        informNeighboursAboutNewBlockedCell(i)

  def informNeigboursAboutNewFreeCell(cell):
    neighbours = getNeighbours(cell)
    for i in neighbours:
      if i.visited:
        updateKnownNeighbours(i)


  def informNeighboursAboutNewBlockedCell(cell):
    neighbours = getNeighbours(cell)
    for i in neighbours:
      if i.visited:
        updateKnownBlockedNeighbours(i)

  # this is the planning phase
  def AStar(knowledgeGrid, start):
    queue = PriorityQueue()
    closedMap = {}
    closedMap[(start.x, start.y)] = start
    queue.put((start.total_cost,start))    # first value in the tuple is priority(cost), second is the cell object
    goal = knowledgeGrid.goal

    while not queue.empty():   #while queue is not empty
      getter = queue.get()
      current = getter[1]
      if isCellOpen(current,n, knowledgeGrid) is False and isCellWithinGrid(current.x, current.y, n) is True:
        knowledgeGrid.world[current.x, current.y] = 1

      if current.x == goal.x and current.y == goal.y:
        path = reconstructPath(current, start, closedMap)
        return executePlannedPath(path)
    
      rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))     # right neighbour
      downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0, (current.x, current.y))    # down neighbour
      leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))  # left neighbour
      topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0, (current.x, current.y))  # top neighbour
     
      neighbors = [rightCell, downCell, leftCell, topCell] # calculate the f(n) for each neighbour of current

      for i in neighbors:
        i.total_cost = calculatefn(i)         # calculate the f(n) for each neighbour of current
        if isCellOpen(i,n, knowledgeGrid):
          element = closedMap.get((i.x, i.y))
          if element is not None:
            ''' 
            If new path to neigbour costs less than previous path to it, then update cell.
            This cell has already been visited before, but we are just checking
            if the current path is less costlier than the one visited before
            '''
            if i.total_cost < closedMap[(i.x, i.y)].total_cost:
             closedMap[(i.x, i.y)] = i

          else:
           closedMap[(i.x, i.y)] = i      # this cell is being added first time
           queue.put((i.total_cost, i))
      previous = current
    # queue is empty but goal node has not been reached
    return False

  def reconstructPath(current, start, closedMap):
    path = collections.deque()
    
    while True:
      if (current.x, current.y) == (start.x, start.y):
        break
      path.appendleft(current)
      current = closedMap[(current.parent[0], current.parent[1])]
    
    return path

  def executePlannedPath(path):
    grid = gridObj.world
    goal = gridObj.goal

    for index, current in enumerate(path):
      alreadyConfirmed = False
      if knowledgeGrid.cellMap.get((current.x, current.y)) is None:
        knowledgeGrid.cellMap[(current.x, current.y)] = current

      else:
        alreadyConfirmed = True
        current = knowledgeGrid.cellMap[(current.x, current.y)]

      if current.x == goal.x and current.y == goal.y:
        return knowledgeGrid.path

      # hit blocked cell
      if grid[current.x, current.y] == 1:
        current.confirmed = True
        knowledgeGrid.world[current.x,current.y] = 1

        #setting restart point  
        knowledgeGrid.start = gridObj.cellMap[current.parent[0], current.parent[1]]

        #adding to the cell map
        knowledgeGrid.cellMap[(current.x, current.y)] = current

        # inform the neighbours that this is blocked
        if alreadyConfirmed == False: 
          informNeighboursAboutNewBlockedCell(current)
        return None

      else:
        # can only sense neighbours of free cells
        current.confirmed = True
        current.visited = True
        current.total_neighbours = gridObj.cellMap[(current.x, current.y)].total_neighbours
        current.total_blocked_neighbours = gridObj.cellMap[(current.x, current.y)].total_blocked_neighbours

        if alreadyConfirmed == False:
          current = getKnownNeighbours(current)

        knowledgeGrid.cellMap[(current.x, current.y)] = current
        knowledgeGrid.path.append(current)
        inferenceEngine(current)

        # inform the neighbours that this is free
        if alreadyConfirmed == False:
          informNeigboursAboutNewFreeCell(current)

  def setRestartPoint(restartPoint, restartValue):
    for i in range(0, restartValue, 1):
      restartPoint = knowledgeGrid.cellMap[(restartPoint.parent[0], restartPoint.parent[1])]

    knowledgeGrid.start = restartPoint
    return
  
  def countBlockedNeighboursInRealGrid():
    # populate the information about blocked neighbours in the real grid
    for i in range(n):
      for j in range(n):
        cell = Cell(i, j, 0, 0, (0,0)) 
        cell = senseNeighbours(cell)

        # create object for all cells in the grid
        gridObj.cellMap[(cell.x,cell.y)] = cell

  def repeatedAStar():
    countBlockedNeighboursInRealGrid()

    #mark start as confirmed
    knowledgeGrid.start.confirmed = True
    knowledgeGrid.start.total_blocked_neighbours = gridObj.cellMap[(0,0)].total_blocked_neighbours
    knowledgeGrid.cellMap[(0,0)] = knowledgeGrid.start

    while True:
        if len(knowledgeGrid.path) > 0:
          previousStartPoint = knowledgeGrid.path[len(knowledgeGrid.path)-1]

          # if we are restarting from the same point where we restarted previously, then dont add this to the path again
          if knowledgeGrid.start.x != previousStartPoint.x and knowledgeGrid.start.y != previousStartPoint.y:
            knowledgeGrid.path.append(knowledgeGrid.start)

        path = AStar(knowledgeGrid, knowledgeGrid.start)
        if path is None:
          continue

        elif path == False:
          return False

        else:
          knowledgeGrid.world[0, 0] = 5
          knowledgeGrid.world[knowledgeGrid.start.x, knowledgeGrid.start.y] = 5
          
          for i in knowledgeGrid.path:
            knowledgeGrid.world[i.x, i.y] = 5

          knowledgeGrid.world[knowledgeGrid.goal.x, knowledgeGrid.goal.y] = 5
          print(knowledgeGrid.world)
          return True

  return repeatedAStar()

PartialSensing(10, 0.3)

[[5 0 0 0 0 0 0 0 0 0]
 [5 5 0 0 0 0 0 0 0 0]
 [1 5 5 0 0 0 0 0 0 0]
 [0 1 5 0 0 0 0 0 0 0]
 [0 0 5 5 5 1 0 0 0 0]
 [0 0 0 5 1 0 0 0 0 0]
 [0 0 0 5 0 0 0 0 0 0]
 [0 0 0 5 0 0 0 0 0 0]
 [0 0 0 5 5 0 0 0 0 0]
 [0 0 0 0 5 5 5 5 5 5]]


True

# Neural network to mimic Agent 1

1. Find out how to look inside the history object

In [ ]:
#return 2 tuples
GridTest['Grid']
#MovementDict['Grid']

#def load_data():
#  MovementFrame[0]

#80-20 split for train and test
#(x_train, y_train ), ( x_test, y_test ) = tf.keras.datasets.mnist.load_data()

train_in = x_train
train_out = y_train
print("Train in")
print(train_in)
#test_in = np.reshape( x_test, (-1, 10, 10) )
#train_out = tf.keras.utils.to_categorical( y_train, 4 )
print("Train out")
print(train_out)
print('Train in shape')
print(np.shape(train_in))

print('Train out shape')
print(np.shape(train_out))
#test_out = tf.keras.utils.to_categorical( y_test, 4 )

#define neural network layers, size of each layer (no. of nodes in each layer)
grid_input = tf.keras.layers
flatten_image = tf.keras.layers.Flatten()( grid_input )
#dense_1 is the first hidden layer (fully connected)
dense_1 = tf.keras.layers.Dense( units = 500, activation = tf.nn.relu )( flatten_image )

#dense_2 is the second hidden layer (fully connected)
dense_2 = tf.keras.layers.Dense( units = 500, activation = tf.nn.relu )( dense_1 )

#output layer
movement_output = tf.keras.layers.Dense( units = 4, activation = None )( dense_2 )

#probabilities = tf.keras.layers.Softmax()( logits )

model = tf.keras.Model( inputs = grid_input, outputs = movement_output )
model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

history = model.fit( train_in, train_out, epochs = 20, verbose=0)
#output = model.eval(new_grid)
#print(history)

Train in


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



TypeError: ignored

# Movement by neural network

In [ ]:
#generate random grids
#solve these grids with the neural network
#assume that the grid is solvable. No need to assume, professor told that only work on solvable grids

goal_found_vector = [0, 0, 0, 0]
while predicted_move != goal_found_vector:
  flatten the grid
  feed the first iteration as x_test, get predicted_move
  if predicted_move = [1,0,0,0], move left and so on

Project 2 input
Knowledge grid same as project 1 input, since we dont want to pass on the inferred knowledge
Another grid where every value = no. of sensed blocked neighbours  for each cell

3*3 grid, initially
Start from 0, 0
Move to 0,1
0 0 0
0 0 0
0 0 0

the next grid
2 0 0
1 0 0
0 0 0

First train the NN on knowledge grid
Then train on other grid

Agent 2 makes the decision to move considering all the inferred knowledge
So its output vector is better than agent 1's output vector
The goal of the NN might be to mimic this better movement just by mapping the input space to output space, where the input space does not contain the inferred knowledge, but that inferred knowledge is inherent in the output space, because the output of agent 2 was decided by it considering all the inferred
The output vector for project 2 will

Project 1 Cell Iterations

In [ ]:
#import tensorflow as tf
import collections
from queue import PriorityQueue
import numpy as np
#import pandas as pd
# import matplotlib.pyplot as plt
from math import sqrt
# from ast import literal_eval
MovementDict = {'Grid':[], 'Movement':[]}
def Voyage(n, p):
    class Cell:
        # x = x-coordinate, y = y-coordinate in grid. Values from 0 to n-1
        def __init__(self, x, y, cost_from_start, total_cost, parent):
            self.x = x
            self.y = y
            self.cost_from_start = cost_from_start
            self.total_cost = total_cost
            self.parent = parent

        def __lt__(self, other):  # Added to add an object to the priority queue
            return self.total_cost < other.total_cost

    class Grid:
        def __init__(self, world, start, goal, path):
            self.iterations = 0
            self.world = world
            self.start = start
            self.goal = goal
            self.path = []
            self.restartValue = 0
            self.allCells = {}  # a dictionary to lookup a cell. Used when looking up its gScore

    def gridworld2(n, p):
        start = [0, 0];
        end = [n - 1, n - 1]
        gridworld = []

        for width in range(n):  # Loop creates n lists of n elements
            width = []
            for length in range(n):
                rand = np.random.random()
                if rand > p:
                    width.append(0)
                else:
                    width.append(1)
            gridworld.append(width)

        gridworld = np.array(gridworld)  # List of lists is turned into an array
        gridworld[start] = 0;
        gridworld[end] = 0
        return gridworld

    gridObj = Grid(gridworld2(n, p), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    knowledgeGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    iterationGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), []) # declaration UCode
    def isCellWithinGrid(x, y, n):
        if (x < 0) or (x > (n - 1)) or (y < 0) or (y > (n - 1)):
            return False

    def isCellOpen(cell, n, grid):
        if isCellWithinGrid(cell.x, cell.y, n) == False:  # check if within gridworld bounds
            return False

        if grid[cell.x][cell.y] == 1:  # check if blocked
            return False

        return True

    def calculatefn(cell):
        goal = (n - 1, n - 1)
        estimate = calculateManhattanDistance(cell, goal)
        return estimate + cell.cost_from_start

    def calculateManhattanDistance(cell, goal):
        return abs(cell.x - goal[0]) + abs(cell.y - goal[1])

    # this is the planning phase
    def AStar(currentGrid, start):
        queue = PriorityQueue()
        gridObj.allCells = {}
        gridObj.allCells[(start.x, start.y)] = start
        queue.put((start.total_cost, start))  # first value in the tuple is priority(cost), second is the cell object
        goal = gridObj.goal

        while not queue.empty():  # while queue is not empty
            getter = queue.get()
            current = getter[1]

            if current.x == goal.x and current.y == goal.y:
                path = reconstructPath(current, start)
                return executePlannedPath(path)

            rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0,
                             (current.x, current.y))  # right neighbour
            downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # down neighbour
            leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # left neighbour
            topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0,
                           (current.x, current.y))  # top neighbour

            neighbors = [rightCell, downCell, leftCell, topCell]  # calculate the f(n) for each neighbour of current

            for i in neighbors:
                i.total_cost = calculatefn(i)  # calculate the f(n) for each neighbour of current
                isItOpen = isCellOpen(i, n, currentGrid)
                if isCellOpen(i, n, currentGrid):
                    element = gridObj.allCells.get((i.x, i.y))
                    if element is not None:
                        ''' 
                        If new path to neigbour costs less than previous path to it, then update cell.
                        This cell has already been visited before, but we are just checking
                        if the current path is less costlier than the one visited before
                        '''
                        if i.total_cost < gridObj.allCells[(i.x, i.y)].total_cost:
                            gridObj.allCells[(i.x, i.y)] = i

                    else:
                        gridObj.allCells[(i.x, i.y)] = i  # this cell is being added first time
                        queue.put((i.total_cost, i))

        # queue is empty but goal node has not been reached
        return False

    def reconstructPath(current, start):
        path = collections.deque()

        while True:
            if (current.x, current.y) == (start.x, start.y):
                break
            path.appendleft(current)
            current = gridObj.allCells[(current.parent[0], current.parent[1])]

        return path

    def directionArray(cell):
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (0, 1):
            return [1, 0, 0, 0]  # Right
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (0, -1):
            return [0, 1, 0, 0]  # Left
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (-1, 0):
            return [0, 0, 1, 0]  # Up
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (1, 0):
            return [0, 0, 0, 1]  # Down

    def executePlannedPath(path):
        grid = gridObj.world
        goal = gridObj.goal
        isPathFeasible = True
        iterationGrid.world[0,0] = 1
        for index, current in enumerate(path):
            print("Visiting " +str(current.x)+'/'+str(current.y))
            iterationGrid.world[current.x, current.y] += 1 #Ucode iteration assingment
            print('Cell Iterations :')
            print(iterationGrid.world)
            gridObj.iterations += 1

            # print(current.x-current.parent[0], current.y-current.parent[1])
            MovementDict['Movement'].append(directionArray(current))
            og = knowledgeGrid.world[current.parent[0], current.parent[1]]
            knowledgeGrid.world[current.parent[0], current.parent[1]] = 2
            # print(knowledgeGrid.world)
            MovementDict['Grid'].append(knowledgeGrid.world.flatten().tolist())
            knowledgeGrid.world[current.parent[0], current.parent[1]] = og
            if current.x == goal.x and current.y == goal.y:
                MovementDict['Movement'].append([0, 0, 0, 0])
                og = knowledgeGrid.world[current.x, current.y]
                knowledgeGrid.world[current.x, current.y] = 2
                # print(knowledgeGrid.world)
                MovementDict['Grid'].append(knowledgeGrid.world.flatten().tolist())
                knowledgeGrid.world[current.x, current.y] = og
                return gridObj.path

            if grid[current.x][current.y] == 1:
                knowledgeGrid.world[current.x][current.y] = 1
                isPathFeasible = False

                # this should be the restart point now
                setRestartPoint(gridObj.allCells[current.parent[0], current.parent[1]], 0)
                print('Restarting point'+str(current.parent[0])+'/'+str(current.parent[1]))
                iterationGrid.world[current.parent[0], current.parent[1]] += 1  # Ucode iteration assingment
                return None

            else:
                gridObj.path.append(current)

            # update the neighbours in the actual grid
            # update the blockedNeighboursCount
            if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
                knowledgeGrid[current.x + 1, current.y] = 1

            if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
                knowledgeGrid[current.x][current.y + 1] = 1

            if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x - 1][current.y] == 1:
                knowledgeGrid[current.x + 1][current.y] = 1

            if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x][current.y - 1] == 1:
                knowledgeGrid[current.x + 1][current.y] = 1

    def setRestartPoint(restartPoint, restartValue):
        for i in range(0, restartValue, 1):
            restartPoint = gridObj.allCells[(restartPoint.parent[0], restartPoint.parent[1])]

        gridObj.start = restartPoint
        return

    def restartController(current):
        # in a loop run the restart again and again
        # starting from the current cell's parent to further up
        while (current.x != 0 and current.y != 0):
            current = gridObj.allCells[(current.parent[0], current.parent[1])]
            gridObj.start = current

    def repeatedAStar():
        while True:
            if len(gridObj.path) > 0:
                previousStartPoint = gridObj.path[len(gridObj.path) - 1]

                # if we are restarting from the same point where we restarted previously, then dont add this to the path again
                if gridObj.start.x != previousStartPoint.x and gridObj.start.y != previousStartPoint.y:
                    gridObj.path.append(gridObj.start)

            path = AStar(knowledgeGrid.world, gridObj.start)
            if path is None:
                continue

            elif path == False:
                return False

            # final path to goal found
            else:
                gridObj.world[0][0] = 5
                gridObj.world[gridObj.start.x][gridObj.start.y] = 5

                for i in gridObj.path:
                    gridObj.world[i.x][i.y] = 5

                gridObj.world[gridObj.goal.x][gridObj.goal.y] = 5
                print(gridObj.world)
                print("Total iterations = ", gridObj.iterations)

                return True

    def restartAtDifferentPoints():
        for i in range(0, (n * n - 1)):
            repeatedAStar()
        return

    return repeatedAStar()


Voyage(5, 0.4)

Visiting 1/0
Cell Iterations :
[[1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
Restarting point0/0
Visiting 0/1
Cell Iterations :
[[1 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
Visiting 1/1
Cell Iterations :
[[1 1 0 0 0]
 [1 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
Visiting 2/1
Cell Iterations :
[[1 1 0 0 0]
 [1 1 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
Restarting point1/1
Visiting 1/2
Cell Iterations :
[[1 1 0 0 0]
 [1 2 1 0 0]
 [0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
Visiting 2/2
Cell Iterations :
[[1 1 0 0 0]
 [1 2 1 0 0]
 [0 1 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
Visiting 3/2
Cell Iterations :
[[1 1 0 0 0]
 [1 2 1 0 0]
 [0 1 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]]
Visiting 4/2
Cell Iterations :
[[1 1 0 0 0]
 [1 2 1 0 0]
 [0 1 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]]
Visiting 4/3
Cell Iterations :
[[1 1 0 0 0]
 [1 2 1 0 0]
 [0 1 1 0 0]
 [0 0 1 0 0]
 [0 0 1 1 0]]
Visiting 4/4
Cell Iterations :
[[1 1 0 0 0]
 [1 2 1 0 0]
 [0 1 1 0 0]
 [0 0 1 0 0]
 [0 

True

Agent 5

In [ ]:
import collections
from queue import PriorityQueue
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from math import sqrt
from timeit import default_timer as timer
import random
import csv
import xlsxwriter
MovementDict = {'Grid':[],'Movement':[],'MinusInferance':[],'InferanceGrid':[]}

def PartialSensing(n, p):
    class Solver:
        def __init__(self):
            # a variable is a cell
            # constraints are all the equations that this cell is part of
            self.variables_to_constraints_map = {}

            # each constraint will a key value pair
            # key is a tuple, blocked_neighbours is a value
            # this is a list of constraints
            self.constraints = []

        def add_constraint(self, variable, n_list, value):
            variable_set = set()
            for i in n_list:
                print('constraint')
                print(i.x, i.y)
                variable_set.add((i.x * n + i.y))

            constraint = {}
            constraint[tuple(variable_set)] = value
            self.constraints.append(constraint)
            constraintIndex = len(self.constraints) - 1

            # adding constraint
            for v in variable_set:
                if self.variables_to_constraints_map.get(v) is None:
                    self.variables_to_constraints_map[v] = []
                self.variables_to_constraints_map[v].append(constraintIndex)

        def check_satisfaction_conditions(self, lhs, rhs):
            if rhs == 0:
                # all cells on lhs are free
                for i in range(0, len(lhs), 1):
                    cellX, cellY = divmod(lhs[i], n)
                    inferred_free(cellX, cellY)

            elif len(lhs) == rhs:
                # all cells on lhs are blocked
                for i in range(0, len(lhs), 1):
                    cellX, cellY = divmod(lhs[i], n)
                    inferred_blocked(cellX, cellY)

        def solve_constraint(self, cell, cell_value):
            print("Solving constraint for ", cell.x, cell.y)
            # Solved when:
            # RHS becomes zero means all are free
            # Size of the list/set on left side = no. on the right side means all are blocked or all are free
            variable = cell.x * n + cell.y
            # get all constraint equations of this variable
            if self.variables_to_constraints_map.get(variable) is None:
                print("No constraint exists for ", cell.x, cell.y)
                return

            constraints = self.variables_to_constraints_map[variable]
            for constraint_index in constraints:
                constraint = self.constraints[constraint_index]
                (lhs, rhs), = constraint.items()
                temp_lhs = list(lhs)
                temp_lhs.remove(variable)
                lhs = tuple(temp_lhs)

                if cell_value == 1:
                    rhs -= 1

                self.variables_to_constraints_map[variable].remove(constraint_index)
                constraint = {}
                constraint[lhs] = rhs
                self.constraints[constraint_index] = constraint
                self.check_satisfaction_conditions(lhs, rhs)
            return

    solver = Solver()

    def inferred_free(x, y):
        if knowledgeGrid.cellMap[(x, y)].confirmed:
            return
        knowledgeGrid.world[x, y] = 0
        knowledgeGrid.cellMap[(x, y)].confirmed = True
        # print("Inferred ", x, y, " as free")
        # print("Reality says ", x, y, " is", gridObj.world[x, y])
        if knowledgeGrid.world[x, y] != gridObj.world[x, y]:
            print("Wrong inference")
        # inform neighbours that this cell has been free
        informNeigboursAboutNewFreeCell(knowledgeGrid.cellMap[(x, y)])
        solver.solve_constraint(knowledgeGrid.cellMap[(x, y)], 0)

    def inferred_blocked(x, y):
        if knowledgeGrid.cellMap[(x, y)].confirmed:
            return

        knowledgeGrid.world[x, y] = 1
        knowledgeGrid.cellMap[(x, y)].confirmed = True
        # print("Inferred ", x, y, " as blocked")
        # print("Reality says ", x, y, " is", gridObj.world[x, y])
        if knowledgeGrid.world[x, y] != gridObj.world[x, y]:
            print("Wrong inference")
        # inform neighbours that this cell has been free
        informNeighboursAboutNewBlockedCell(knowledgeGrid.cellMap[(x, y)])
        solver.solve_constraint(knowledgeGrid.cellMap[(x, y)], 1)

    def agent_four_inference(cell, unknown_blocked):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            if i.confirmed:
                if knowledgeGrid.world[i.x, i.y] == 1:
                    unknown_blocked -= 1
                # remove this from the equation because its value is known
                # if the value was 1, then that case was handled in the above line
                # if value was 0, then there is no effect on RHS
                neighbours.remove(i)

        solver.add_constraint(cell.x * n + cell.y, neighbours, unknown_blocked)
        return

    class Cell:
        # x = x-coordinate, y = y-coordinate in grid. Values from 0 to n-1
        def __init__(self, x, y, cost_from_start, total_cost, parent):
            self.x = x
            self.y = y
            self.cost_from_start = cost_from_start
            self.total_cost = total_cost
            self.total_neighbours = 0
            self.total_blocked_neighbours = 0
            self.known_neighbours = 0
            self.known_blocked_neighbours = 0
            self.parent = parent
            self.confirmed = False
            self.visited = False

        def __lt__(self, other):  # Added to add an object to the priority queue
            return self.total_cost < other.total_cost

    class Grid:
        def __init__(self, world, start, goal, path):
            self.world = world
            self.start = start
            self.goal = goal
            self.path = []
            self.cellMap = {}  # a dictionary to lookup a cell.

    def gridworld2(n, p):
        start = [0, 0];
        end = [n - 1, n - 1]
        gridworld = []

        for width in range(n):  # Loop creates n lists of n elements
            width = []
            for length in range(n):
                rand = np.random.random()
                if rand > p:
                    width.append(0)
                else:
                    width.append(1)
            gridworld.append(width)

        gridworld = np.array(gridworld)  # List of lists is turned into an array
        gridworld[start] = 0;
        gridworld[end] = 0
        return gridworld

    gridObj = Grid(gridworld2(n, p), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    knowledgeGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    KnowledgeGridMinusInferance = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), []) #declaration
    InferanceGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), []) #declaration
    def isCellWithinGrid(x, y, n):
        if (x < 0) or (x > (n - 1)) or (y < 0) or (y > (n - 1)):
            return False
        return True

    def isCellOpen(cell, n, grid):
        if isCellWithinGrid(cell.x, cell.y, n) == False:  # check if within gridworld bounds
            return False

        if grid.world[cell.x, cell.y] == 1:  # check if blocked
            return False

        return True

    def calculatefn(cell):
        goal = (n - 1, n - 1)
        estimate = calculateManhattanDistance(cell, goal)
        return estimate + cell.cost_from_start

    # Probability Start
    def calculateProbilityfn(cell):
        goal = (n - 1, n - 1)
        print()
        print('Cell location probability function', cell.x, cell.y)
        print('cell.total_blocked_neighbours - cell.known_blocked_neighbours): ' +
              str(cell.total_blocked_neighbours) + ' - ' + str(cell.known_blocked_neighbours) + ' = ' + str(
            cell.total_blocked_neighbours - cell.known_blocked_neighbours))
        print('cell.total_neighbours:' + str(cell.total_neighbours) + ' - ' + ' cell.known_neighbours :' + str(
            cell.known_neighbours) + ' = ' + str(cell.total_neighbours - cell.known_neighbours))
        print()
        print('Manhattan: ' + str(calculateManhattanDistance(cell, goal)))

        # when some neighbours are unknown
        if (cell.total_neighbours - cell.known_neighbours) != 0:
            estimateWithProbability = round(calculateManhattanDistance(cell, goal) * (((
                                                                                                   cell.total_blocked_neighbours - cell.known_blocked_neighbours) / (
                                                                                                   cell.total_neighbours - cell.known_neighbours)) + 1))

        # when all neighbours of a cell are known, no need to involve probabilistic calculations for them
        else:
            estimateWithProbability = calculateManhattanDistance(cell, goal)
        print('Estimate : ' + str(estimateWithProbability))
        return estimateWithProbability + cell.cost_from_start
        # Probability End

    def calculateManhattanDistance(cell, goal):
        return abs(cell.x - goal[0]) + abs(cell.y - goal[1])

    def getNeighbours(cell):
        # east, south, west, north, ne, se, sw, nw in that order
        x = [1, 0, -1, 0, 1, 1, -1, -1]
        y = [0, 1, 0, -1, -1, 1, 1, -1]

        neighbours = []
        for i in range(8):
            if isCellWithinGrid(cell.x + x[i], cell.y + y[i], n):
                if knowledgeGrid.cellMap.get((cell.x + x[i], cell.y + y[i])) is None:
                    knowledgeGrid.cellMap[(cell.x + x[i], cell.y + y[i])] = Cell(cell.x + x[i], cell.y + y[i], 0, 0,
                                                                                 (cell.x, cell.y))
                neighbours.append(knowledgeGrid.cellMap[(cell.x + x[i], cell.y + y[i])])

        return neighbours

    def senseNeighbours(cell):
        ECell = Cell(cell.x + 1, cell.y, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # East neighbour
        SCell = Cell(cell.x, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South neighbour
        WCell = Cell(cell.x - 1, cell.y, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # West neighbour
        NCell = Cell(cell.x, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North neighbour
        NECell = Cell(cell.x + 1, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North East neighbour
        SECell = Cell(cell.x + 1, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South East neighbour
        SWCell = Cell(cell.x - 1, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South West neighbour
        NWCell = Cell(cell.x - 1, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North West neighbour

        potential_neighbours = [ECell, SCell, WCell, NCell, NECell, SECell, SWCell, NWCell]
        neighbours = [i for i in potential_neighbours if isCellWithinGrid(i.x, i.y, n)]
        total = 0
        blocked = 0
        for i in neighbours:
            total += 1
            if gridObj.world[i.x, i.y] == 1:
                blocked += 1
        cell.total_neighbours = total
        cell.total_blocked_neighbours = blocked
        return cell

    def getKnownNeighbours(cell):
        # east, south, west, north, ne, se, sw, nw in that order
        x = [1, 0, -1, 0, 1, 1, -1, -1]
        y = [0, 1, 0, -1, -1, 1, 1, -1]

        neighbours = []
        for i in range(8):
            if isCellWithinGrid(cell.x + x[i], cell.y + y[i], n):
                neighbourCell = knowledgeGrid.cellMap.get((cell.x + x[i], cell.y + y[i]))
                if neighbourCell is not None and neighbourCell.confirmed:
                    cell.known_neighbours += 1
                    if knowledgeGrid.world[cell.x + x[i], cell.y + y[i]] == 1:
                        cell.known_blocked_neighbours += 1
        return cell

    def updateKnownNeighbours(cell):
        knowledgeGrid.cellMap[(cell.x, cell.y)].known_neighbours += 1
        inferenceEngine(cell)

    def updateKnownBlockedNeighbours(cell):
        cell.known_blocked_neighbours += 1
        cell.known_neighbours += 1
        knowledgeGrid.cellMap[(cell.x, cell.y)] = cell
        inferenceEngine(cell)

    def inferenceEngine(cell):
        if cell.total_blocked_neighbours == cell.known_blocked_neighbours:
            markRemainingNeighboursAsFree(cell)

        # if totalFreeNeighbours = knownFreeNeighbours
        if (cell.total_neighbours - cell.total_blocked_neighbours
                == (cell.known_neighbours - cell.known_blocked_neighbours)):
            markRemainingNeighboursAsBlocked(cell)

        # Early Exit Function
        if cell.total_blocked_neighbours == cell.total_neighbours - 1:
            markRemainingNeighboursAsBlocked(cell)

    def markRemainingNeighboursAsFree(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            # this new freely marked cell must inform its neighbours which have been visited
            if i.confirmed == False:
                print("Marking ", i.x, i.y, " as a free neighbour of ", cell.x, cell.y)
                if gridObj.world[i.x, i.y] != 0:
                    print("in marking remaining free, Wrong inference for cell", i.x, i.y, " courtesy of ", cell.x,
                          cell.y)
                    print("Total neighbours:", cell.total_neighbours)
                    print("Total blocked neighbours:", cell.total_blocked_neighbours)
                    print("Known blocked neighbours:", cell.known_blocked_neighbours)
                    print("Known neighbours:", cell.known_neighbours)
                knowledgeGrid.world[i.x, i.y] = 0
                cell.known_neighbours += 1
                knowledgeGrid.cellMap[(i.x, i.y)] = cell
                i.confirmed = True
                knowledgeGrid.cellMap[(i.x, i.y)] = i
                # inform the neighbours
                informNeigboursAboutNewFreeCell(i)
                solver.solve_constraint(knowledgeGrid.cellMap[(i.x, i.y)], 0)

    def markRemainingNeighboursAsBlocked(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            # this new blocked cell must inform its neighbours which have been visited
            if i.confirmed == False:
                print("Marking ", i.x, i.y, " as a blocked neighbour of ", cell.x, cell.y)
                cell.known_blocked_neighbours += 1
                cell.known_neighbours += 1
                knowledgeGrid.cellMap[(i.x, i.y)] = cell
                if gridObj.world[i.x, i.y] != 1:
                    print("in marking remaining blocked, Wrong inference for cell", i.x, i.y, " courtesy of ", cell.x,
                          cell.y)
                    print("Total neighbours:", cell.total_neighbours)
                    print("Total blocked neighbours:", cell.total_blocked_neighbours)
                    print("Known blocked neighbours:", cell.known_blocked_neighbours)
                    print("Known neighbours:", cell.known_neighbours)
                knowledgeGrid.world[i.x, i.y] = 1
                i.confirmed = True
                knowledgeGrid.cellMap[(i.x, i.y)] = i
                informNeighboursAboutNewBlockedCell(i)
                solver.solve_constraint(knowledgeGrid.cellMap[(i.x, i.y)], 1)

    def informNeigboursAboutNewFreeCell(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            if i.visited:
                print(cell.x, cell.y, " is a known free neighbour of ", i.x, i.y)
                updateKnownNeighbours(i)

    def informNeighboursAboutNewBlockedCell(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            if i.visited:
                print(cell.x, cell.y, " is a known blocked neighbour of ", i.x, i.y)
                updateKnownBlockedNeighbours(i)

    # this is the planning phase
    def AStar(knowledgeGrid, start):
        print("Starting from ", start.x, start.y)
        queue = PriorityQueue()
        closedMap = {}
        closedMap[(start.x, start.y)] = start
        queue.put((start.total_cost, start))  # first value in the tuple is priority(cost), second is the cell object
        goal = knowledgeGrid.goal

        while not queue.empty():  # while queue is not empty
            getter = queue.get()
            current = getter[1]
            if isCellOpen(current, n, knowledgeGrid) is False and isCellWithinGrid(current.x, current.y, n) is True:
                knowledgeGrid.world[current.x, current.y] = 1

            if current.x == goal.x and current.y == goal.y:
                path = reconstructPath(current, start, closedMap)
                return executePlannedPath(path)

            rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0,
                             (current.x, current.y))  # right neighbour
            downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # down neighbour
            leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # left neighbour
            topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0,
                           (current.x, current.y))  # top neighbour

            neighbors = [rightCell, downCell, leftCell, topCell]  # calculate the f(n) for each neighbour of current

            for i in neighbors:
                # i.total_cost = calculatefn(i)  # calculate the f(n) for each neighbour of current

                if knowledgeGrid.cellMap.get((i.x, i.y)) and knowledgeGrid.cellMap[(i.x, i.y)].visited:
                    i.total_cost = calculateProbilityfn(knowledgeGrid.cellMap[(i.x, i.y)])
                else:
                    i.total_cost = calculatefn(i)

                if isCellOpen(i, n, knowledgeGrid):
                    element = closedMap.get((i.x, i.y))
                    if element is not None:
                        ''' 
                        If new path to neigbour costs less than previous path to it, then update cell.
                        This cell has already been visited before, but we are just checking
                        if the current path is less costlier than the one visited before
                        '''
                        if i.total_cost < closedMap[(i.x, i.y)].total_cost:
                            closedMap[(i.x, i.y)] = i

                    else:
                        closedMap[(i.x, i.y)] = i  # this cell is being added first time
                        queue.put((i.total_cost, i))
            previous = current
        # queue is empty but goal node has not been reached
        return False

    def reconstructPath(current, start, closedMap):
        path = collections.deque()

        while True:
            if (current.x, current.y) == (start.x, start.y):
                break
            path.appendleft(current)
            current = closedMap[(current.parent[0], current.parent[1])]

        return path

    def directionArray(cell):
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (0, 1):
            return [1, 0, 0, 0]  # Right
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (0, -1):
            return [0, 1, 0, 0]  # Left
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (-1, 0):
            return [0, 0, 1, 0]  # Up
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (1, 0):
            return [0, 0, 0, 1]  # Down

    def executePlannedPath(path):
        grid = gridObj.world
        goal = gridObj.goal

        print('total blocked at 00   ' + str(gridObj.cellMap[0, 0].total_blocked_neighbours))
        InferanceGrid.world[0, 0] = gridObj.cellMap[0, 0].total_blocked_neighbours

        for index, current in enumerate(path):
            # Ucode Start
            MovementDict['Grid'].append(knowledgeGrid.world.flatten().tolist())
            MovementDict['Movement'].append(directionArray(current))
            MovementDict['MinusInferance'].append(KnowledgeGridMinusInferance.world.flatten().tolist())
            MovementDict['InferanceGrid'].append(InferanceGrid.world.flatten().tolist())

            # print('actual values')
            # print(current.x, current.y)
            # print('Grid Minus Inferance:')
            #
            # print(KnowledgeGridMinusInferance.world)
            # MovementDict['InferanceGrid'].append(KnowledgeGridMinusInferance.world.flatten().tolist())
            # print('Inferance Values')
            # print(InferanceGrid.world.flatten().tolist())

            # Ucode End
            alreadyConfirmed = False
            if knowledgeGrid.cellMap.get((current.x, current.y)) is None:
                knowledgeGrid.cellMap[(current.x, current.y)] = current

            else:
                alreadyConfirmed = True
                current = knowledgeGrid.cellMap[(current.x, current.y)]

            if current.x == goal.x and current.y == goal.y:
                return knowledgeGrid.path

            # hit blocked cell
            if grid[current.x, current.y] == 1:
                current.confirmed = True
                print("Blocked by ", current.x, current.y)
                knowledgeGrid.world[current.x, current.y] = 1

                KnowledgeGridMinusInferance.world[current.x, current.y] = 1

                # setting restart point
                knowledgeGrid.start = gridObj.cellMap[current.parent[0], current.parent[1]]

                KnowledgeGridMinusInferance.world[current.parent[0], current.parent[1]]= 2

                # adding to the cell map
                knowledgeGrid.cellMap[(current.x, current.y)] = current

                # inform the neighbours that this is blocked
                if alreadyConfirmed == False:
                    informNeighboursAboutNewBlockedCell(current)

                solver.solve_constraint(current, 1)

                return None

            else:
                # can only sense neighbours of free cells
                current.confirmed = True
                current.visited = True
                current.total_neighbours = gridObj.cellMap[(current.x, current.y)].total_neighbours
                current.total_blocked_neighbours = gridObj.cellMap[(current.x, current.y)].total_blocked_neighbours
                InferanceGrid.world[(current.x, current.y)] = gridObj.cellMap[(current.x, current.y)].total_blocked_neighbours
                if alreadyConfirmed == False:
                    current = getKnownNeighbours(current)

                knowledgeGrid.cellMap[(current.x, current.y)] = current
                knowledgeGrid.path.append(current)
                inferenceEngine(current)

                # inform the neighbours that this is free
                if alreadyConfirmed == False:
                    informNeigboursAboutNewFreeCell(current)

                # advanced inference-
                agent_four_inference(current, current.total_blocked_neighbours)
                solver.solve_constraint(current, 0)


    def countBlockedNeighboursInRealGrid():
        # populate the information about blocked neighbours in the real grid
        for i in range(n):
            for j in range(n):
                cell = Cell(i, j, 0, 0, (0, 0))
                cell = senseNeighbours(cell)

                # create object for all cells in the grid
                gridObj.cellMap[(cell.x, cell.y)] = cell

    def repeatedAStar():
        countBlockedNeighboursInRealGrid()

        # mark start as confirmed
        knowledgeGrid.start.confirmed = True
        knowledgeGrid.start.total_blocked_neighbours = gridObj.cellMap[(0, 0)].total_blocked_neighbours
        knowledgeGrid.cellMap[(0, 0)] = knowledgeGrid.start

        print(gridObj.world)
        print()
        agent_four_inference(knowledgeGrid.start, knowledgeGrid.start.total_blocked_neighbours)

        while True:
            if len(knowledgeGrid.path) > 0:
                previousStartPoint = knowledgeGrid.path[len(knowledgeGrid.path) - 1]

                # if we are restarting from the same point where we restarted previously, then dont add this to the path again
                if knowledgeGrid.start.x != previousStartPoint.x and knowledgeGrid.start.y != previousStartPoint.y:
                    knowledgeGrid.path.append(knowledgeGrid.start)

            path = AStar(knowledgeGrid, knowledgeGrid.start)
            if path is None:
                continue

            elif path == False:
                print("Knowledge grid line 279:")
                print(knowledgeGrid.world)
                return False

            else:
                knowledgeGrid.world[0, 0] = 5
                knowledgeGrid.world[knowledgeGrid.start.x, knowledgeGrid.start.y] = 5

                for i in knowledgeGrid.path:
                    print('path')
                    print(i.x, i.y)
                    knowledgeGrid.world[i.x, i.y] = 5

                knowledgeGrid.world[knowledgeGrid.goal.x, knowledgeGrid.goal.y] = 5
                print("Knowledge grid line 291:")
                print(knowledgeGrid.world)
                return True

    return repeatedAStar()



PartialSensing(10, 0.3)

import pandas as pd
# df = pd.DataFrame(MovementDict)
df = pd.DataFrame.from_dict(MovementDict, orient='index')
df = df.transpose()
df.to_excel('your_file.xlsx', index=False)

[[0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 1 0 0]
 [0 1 0 1 0 0 0 0 1 0]
 [0 0 1 0 1 0 0 1 1 1]
 [0 0 0 1 1 0 0 0 0 1]
 [0 0 1 0 1 0 0 0 1 1]
 [0 1 0 0 0 0 0 1 1 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]]

constraint
1 0
constraint
0 1
constraint
1 1
Starting from  0 0
total blocked at 00   2
actual values
1 0
Grid Minus Inferance:
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
Inferance Values
[[2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
Blocked by  1 0
Solving constraint for  1 0
Starting from  0 0
total blocked at 00   2
actual values
0 1
Grid Minus Inferance:
[[2 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0

True

Agent 5(Project 2 Iteration count

In [ ]:
import collections
from queue import PriorityQueue
import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from math import sqrt
# from timeit import default_timer as timer
# import random

def PartialSensing(n, p):
    class Solver:
        def __init__(self):
            # a variable is a cell
            # constraints are all the equations that this cell is part of
            self.variables_to_constraints_map = {}

            # each constraint will a key value pair
            # key is a tuple, blocked_neighbours is a value
            # this is a list of constraints
            self.constraints = []

        def add_constraint(self, variable, n_list, value):
            variable_set = set()
            for i in n_list:
                print(i.x, i.y)
                variable_set.add((i.x * n + i.y))

            constraint = {}
            constraint[tuple(variable_set)] = value
            self.constraints.append(constraint)
            constraintIndex = len(self.constraints) - 1

            # adding constraint
            for v in variable_set:
                if self.variables_to_constraints_map.get(v) is None:
                    self.variables_to_constraints_map[v] = []
                self.variables_to_constraints_map[v].append(constraintIndex)

        def check_satisfaction_conditions(self, lhs, rhs):
            if rhs == 0:
                # all cells on lhs are free
                for i in range(0, len(lhs), 1):
                    cellX, cellY = divmod(lhs[i], n)
                    inferred_free(cellX, cellY)

            elif len(lhs) == rhs:
                # all cells on lhs are blocked
                for i in range(0, len(lhs), 1):
                    cellX, cellY = divmod(lhs[i], n)
                    inferred_blocked(cellX, cellY)

        def solve_constraint(self, cell, cell_value):
            print("Solving constraint for ", cell.x, cell.y)
            # Solved when:
            # RHS becomes zero means all are free
            # Size of the list/set on left side = no. on the right side means all are blocked or all are free
            variable = cell.x * n + cell.y
            # get all constraint equations of this variable
            if self.variables_to_constraints_map.get(variable) is None:
                print("No constraint exists for ", cell.x, cell.y)
                return

            constraints = self.variables_to_constraints_map[variable]
            for constraint_index in constraints:
                constraint = self.constraints[constraint_index]
                (lhs, rhs), = constraint.items()
                temp_lhs = list(lhs)
                temp_lhs.remove(variable)
                lhs = tuple(temp_lhs)

                if cell_value == 1:
                    rhs -= 1

                self.variables_to_constraints_map[variable].remove(constraint_index)
                constraint = {}
                constraint[lhs] = rhs
                self.constraints[constraint_index] = constraint
                self.check_satisfaction_conditions(lhs, rhs)
            return

    solver = Solver()

    def inferred_free(x, y):
        if knowledgeGrid.cellMap[(x, y)].confirmed:
            return
        knowledgeGrid.world[x, y] = 0
        knowledgeGrid.cellMap[(x, y)].confirmed = True
        # print("Inferred ", x, y, " as free")
        # print("Reality says ", x, y, " is", gridObj.world[x, y])
        if knowledgeGrid.world[x, y] != gridObj.world[x, y]:
            print("Wrong inference")
        # inform neighbours that this cell has been free
        informNeigboursAboutNewFreeCell(knowledgeGrid.cellMap[(x, y)])
        solver.solve_constraint(knowledgeGrid.cellMap[(x, y)], 0)

    def inferred_blocked(x, y):
        if knowledgeGrid.cellMap[(x, y)].confirmed:
            return

        knowledgeGrid.world[x, y] = 1
        knowledgeGrid.cellMap[(x, y)].confirmed = True
        # print("Inferred ", x, y, " as blocked")
        # print("Reality says ", x, y, " is", gridObj.world[x, y])
        if knowledgeGrid.world[x, y] != gridObj.world[x, y]:
            print("Wrong inference")
        # inform neighbours that this cell has been free
        informNeighboursAboutNewBlockedCell(knowledgeGrid.cellMap[(x, y)])
        solver.solve_constraint(knowledgeGrid.cellMap[(x, y)], 1)

    def agent_four_inference(cell, unknown_blocked):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            if i.confirmed:
                if knowledgeGrid.world[i.x, i.y] == 1:
                    unknown_blocked -= 1
                # remove this from the equation because its value is known
                # if the value was 1, then that case was handled in the above line
                # if value was 0, then there is no effect on RHS
                neighbours.remove(i)

        solver.add_constraint(cell.x * n + cell.y, neighbours, unknown_blocked)
        return

    class Cell:
        # x = x-coordinate, y = y-coordinate in grid. Values from 0 to n-1
        def __init__(self, x, y, cost_from_start, total_cost, parent):
            self.x = x
            self.y = y
            self.cost_from_start = cost_from_start
            self.total_cost = total_cost
            self.total_neighbours = 0
            self.total_blocked_neighbours = 0
            self.known_neighbours = 0
            self.known_blocked_neighbours = 0
            self.parent = parent
            self.confirmed = False
            self.visited = False

        def __lt__(self, other):  # Added to add an object to the priority queue
            return self.total_cost < other.total_cost

    class Grid:
        def __init__(self, world, start, goal, path):
            self.world = world
            self.start = start
            self.goal = goal
            self.path = []
            self.cellMap = {}  # a dictionary to lookup a cell.

    def gridworld2(n, p):
        start = [0, 0];
        end = [n - 1, n - 1]
        gridworld = []

        for width in range(n):  # Loop creates n lists of n elements
            width = []
            for length in range(n):
                rand = np.random.random()
                if rand > p:
                    width.append(0)
                else:
                    width.append(1)
            gridworld.append(width)

        gridworld = np.array(gridworld)  # List of lists is turned into an array
        gridworld[start] = 0;
        gridworld[end] = 0
        return gridworld

    gridObj = Grid(gridworld2(n, p), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    knowledgeGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    iterationGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])  # declaration UCode
    def isCellWithinGrid(x, y, n):
        if (x < 0) or (x > (n - 1)) or (y < 0) or (y > (n - 1)):
            return False
        return True

    def isCellOpen(cell, n, grid):
        if isCellWithinGrid(cell.x, cell.y, n) == False:  # check if within gridworld bounds
            return False

        if grid.world[cell.x, cell.y] == 1:  # check if blocked
            return False

        return True

    def calculatefn(cell):
        goal = (n - 1, n - 1)
        estimate = calculateManhattanDistance(cell, goal)
        return estimate + cell.cost_from_start

    # Probability Start
    def calculateProbilityfn(cell):
        goal = (n - 1, n - 1)
        print()
        print('Cell location probability function', cell.x, cell.y)
        print('cell.total_blocked_neighbours - cell.known_blocked_neighbours): ' +
              str(cell.total_blocked_neighbours) + ' - ' + str(cell.known_blocked_neighbours) + ' = ' + str(
            cell.total_blocked_neighbours - cell.known_blocked_neighbours))
        print('cell.total_neighbours:' + str(cell.total_neighbours) + ' - ' + ' cell.known_neighbours :' + str(
            cell.known_neighbours) + ' = ' + str(cell.total_neighbours - cell.known_neighbours))
        print()
        print('Manhattan: ' + str(calculateManhattanDistance(cell, goal)))

        # when some neighbours are unknown
        if (cell.total_neighbours - cell.known_neighbours) != 0:
            estimateWithProbability = round(calculateManhattanDistance(cell, goal) * (((
                                                                                                   cell.total_blocked_neighbours - cell.known_blocked_neighbours) / (
                                                                                                   cell.total_neighbours - cell.known_neighbours)) + 1))

        # when all neighbours of a cell are known, no need to involve probabilistic calculations for them
        else:
            estimateWithProbability = calculateManhattanDistance(cell, goal)
        print('Estimate : ' + str(estimateWithProbability))
        return estimateWithProbability + cell.cost_from_start
        # Probability End

    def calculateManhattanDistance(cell, goal):
        return abs(cell.x - goal[0]) + abs(cell.y - goal[1])

    def getNeighbours(cell):
        # east, south, west, north, ne, se, sw, nw in that order
        x = [1, 0, -1, 0, 1, 1, -1, -1]
        y = [0, 1, 0, -1, -1, 1, 1, -1]

        neighbours = []
        for i in range(8):
            if isCellWithinGrid(cell.x + x[i], cell.y + y[i], n):
                if knowledgeGrid.cellMap.get((cell.x + x[i], cell.y + y[i])) is None:
                    knowledgeGrid.cellMap[(cell.x + x[i], cell.y + y[i])] = Cell(cell.x + x[i], cell.y + y[i], 0, 0,
                                                                                 (cell.x, cell.y))
                neighbours.append(knowledgeGrid.cellMap[(cell.x + x[i], cell.y + y[i])])

        return neighbours

    def senseNeighbours(cell):
        ECell = Cell(cell.x + 1, cell.y, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # East neighbour
        SCell = Cell(cell.x, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South neighbour
        WCell = Cell(cell.x - 1, cell.y, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # West neighbour
        NCell = Cell(cell.x, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North neighbour
        NECell = Cell(cell.x + 1, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North East neighbour
        SECell = Cell(cell.x + 1, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South East neighbour
        SWCell = Cell(cell.x - 1, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South West neighbour
        NWCell = Cell(cell.x - 1, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North West neighbour

        potential_neighbours = [ECell, SCell, WCell, NCell, NECell, SECell, SWCell, NWCell]
        neighbours = [i for i in potential_neighbours if isCellWithinGrid(i.x, i.y, n)]
        total = 0
        blocked = 0
        for i in neighbours:
            total += 1
            if gridObj.world[i.x, i.y] == 1:
                blocked += 1
        cell.total_neighbours = total
        cell.total_blocked_neighbours = blocked
        return cell

    def getKnownNeighbours(cell):
        # east, south, west, north, ne, se, sw, nw in that order
        x = [1, 0, -1, 0, 1, 1, -1, -1]
        y = [0, 1, 0, -1, -1, 1, 1, -1]

        neighbours = []
        for i in range(8):
            if isCellWithinGrid(cell.x + x[i], cell.y + y[i], n):
                neighbourCell = knowledgeGrid.cellMap.get((cell.x + x[i], cell.y + y[i]))
                if neighbourCell is not None and neighbourCell.confirmed:
                    cell.known_neighbours += 1
                    if knowledgeGrid.world[cell.x + x[i], cell.y + y[i]] == 1:
                        cell.known_blocked_neighbours += 1
        return cell

    def updateKnownNeighbours(cell):
        knowledgeGrid.cellMap[(cell.x, cell.y)].known_neighbours += 1
        inferenceEngine(cell)

    def updateKnownBlockedNeighbours(cell):
        cell.known_blocked_neighbours += 1
        cell.known_neighbours += 1
        knowledgeGrid.cellMap[(cell.x, cell.y)] = cell
        inferenceEngine(cell)

    def inferenceEngine(cell):
        if cell.total_blocked_neighbours == cell.known_blocked_neighbours:
            markRemainingNeighboursAsFree(cell)

        # if totalFreeNeighbours = knownFreeNeighbours
        if (cell.total_neighbours - cell.total_blocked_neighbours
                == (cell.known_neighbours - cell.known_blocked_neighbours)):
            markRemainingNeighboursAsBlocked(cell)

        # Early Exit Function
        if cell.total_blocked_neighbours == cell.total_neighbours - 1:
            markRemainingNeighboursAsBlocked(cell)

    def markRemainingNeighboursAsFree(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            # this new freely marked cell must inform its neighbours which have been visited
            if i.confirmed == False:
                print("Marking ", i.x, i.y, " as a free neighbour of ", cell.x, cell.y)
                if gridObj.world[i.x, i.y] != 0:
                    print("in marking remaining free, Wrong inference for cell", i.x, i.y, " courtesy of ", cell.x,
                          cell.y)
                    print("Total neighbours:", cell.total_neighbours)
                    print("Total blocked neighbours:", cell.total_blocked_neighbours)
                    print("Known blocked neighbours:", cell.known_blocked_neighbours)
                    print("Known neighbours:", cell.known_neighbours)
                knowledgeGrid.world[i.x, i.y] = 0
                cell.known_neighbours += 1
                knowledgeGrid.cellMap[(i.x, i.y)] = cell
                i.confirmed = True
                knowledgeGrid.cellMap[(i.x, i.y)] = i
                # inform the neighbours
                informNeigboursAboutNewFreeCell(i)
                solver.solve_constraint(knowledgeGrid.cellMap[(i.x, i.y)], 0)

    def markRemainingNeighboursAsBlocked(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            # this new blocked cell must inform its neighbours which have been visited
            if i.confirmed == False:
                print("Marking ", i.x, i.y, " as a blocked neighbour of ", cell.x, cell.y)
                cell.known_blocked_neighbours += 1
                cell.known_neighbours += 1
                knowledgeGrid.cellMap[(i.x, i.y)] = cell
                if gridObj.world[i.x, i.y] != 1:
                    print("in marking remaining blocked, Wrong inference for cell", i.x, i.y, " courtesy of ", cell.x,
                          cell.y)
                    print("Total neighbours:", cell.total_neighbours)
                    print("Total blocked neighbours:", cell.total_blocked_neighbours)
                    print("Known blocked neighbours:", cell.known_blocked_neighbours)
                    print("Known neighbours:", cell.known_neighbours)
                knowledgeGrid.world[i.x, i.y] = 1
                i.confirmed = True
                knowledgeGrid.cellMap[(i.x, i.y)] = i
                informNeighboursAboutNewBlockedCell(i)
                solver.solve_constraint(knowledgeGrid.cellMap[(i.x, i.y)], 1)

    def informNeigboursAboutNewFreeCell(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            if i.visited:
                print(cell.x, cell.y, " is a known free neighbour of ", i.x, i.y)
                updateKnownNeighbours(i)

    def informNeighboursAboutNewBlockedCell(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            if i.visited:
                print(cell.x, cell.y, " is a known blocked neighbour of ", i.x, i.y)
                updateKnownBlockedNeighbours(i)

    # this is the planning phase
    def AStar(knowledgeGrid, start):
        print("Starting from ", start.x, start.y)
        queue = PriorityQueue()
        closedMap = {}
        closedMap[(start.x, start.y)] = start
        queue.put((start.total_cost, start))  # first value in the tuple is priority(cost), second is the cell object
        goal = knowledgeGrid.goal

        while not queue.empty():  # while queue is not empty
            getter = queue.get()
            current = getter[1]
            if isCellOpen(current, n, knowledgeGrid) is False and isCellWithinGrid(current.x, current.y, n) is True:
                knowledgeGrid.world[current.x, current.y] = 1

            if current.x == goal.x and current.y == goal.y:
                path = reconstructPath(current, start, closedMap)
                return executePlannedPath(path)

            rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0,
                             (current.x, current.y))  # right neighbour
            downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # down neighbour
            leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # left neighbour
            topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0,
                           (current.x, current.y))  # top neighbour

            neighbors = [rightCell, downCell, leftCell, topCell]  # calculate the f(n) for each neighbour of current

            for i in neighbors:
                # i.total_cost = calculatefn(i)  # calculate the f(n) for each neighbour of current

                if knowledgeGrid.cellMap.get((i.x, i.y)) and knowledgeGrid.cellMap[(i.x, i.y)].visited:
                    i.total_cost = calculateProbilityfn(knowledgeGrid.cellMap[(i.x, i.y)])
                else:
                    i.total_cost = calculatefn(i)

                if isCellOpen(i, n, knowledgeGrid):
                    element = closedMap.get((i.x, i.y))
                    if element is not None:
                        ''' 
                        If new path to neigbour costs less than previous path to it, then update cell.
                        This cell has already been visited before, but we are just checking
                        if the current path is less costlier than the one visited before
                        '''
                        if i.total_cost < closedMap[(i.x, i.y)].total_cost:
                            closedMap[(i.x, i.y)] = i

                    else:
                        closedMap[(i.x, i.y)] = i  # this cell is being added first time
                        queue.put((i.total_cost, i))
            previous = current
        # queue is empty but goal node has not been reached
        return False

    def reconstructPath(current, start, closedMap):
        path = collections.deque()

        while True:
            if (current.x, current.y) == (start.x, start.y):
                break
            path.appendleft(current)
            current = closedMap[(current.parent[0], current.parent[1])]

        return path

    def executePlannedPath(path):
        for i in path:
            print(i.x, i.y)

        grid = gridObj.world
        goal = gridObj.goal
        iterationGrid.world[0, 0] = 1 # Ucode iteration assingment
        for index, current in enumerate(path):
            print("Visiting " + str(current.x) + '/' + str(current.y))  # Ucode iteration assingment
            iterationGrid.world[current.x, current.y] += 1  # Ucode iteration assingment
            print('Cell Iterations :')  # Ucode iteration assingment
            print(iterationGrid.world)  # Ucode iteration assingment
            alreadyConfirmed = False
            if knowledgeGrid.cellMap.get((current.x, current.y)) is None:
                knowledgeGrid.cellMap[(current.x, current.y)] = current

            else:
                alreadyConfirmed = True
                current = knowledgeGrid.cellMap[(current.x, current.y)]

            if current.x == goal.x and current.y == goal.y:
                return knowledgeGrid.path

            # hit blocked cell
            if grid[current.x, current.y] == 1:
                iterationGrid.world[current.parent[0], current.parent[1]] += 1  # Ucode iteration assingment
                current.confirmed = True
                print("Blocked by ", current.x, current.y)
                knowledgeGrid.world[current.x, current.y] = 1

                # setting restart point
                knowledgeGrid.start = gridObj.cellMap[current.parent[0], current.parent[1]]

                # adding to the cell map
                knowledgeGrid.cellMap[(current.x, current.y)] = current

                # inform the neighbours that this is blocked
                if alreadyConfirmed == False:
                    informNeighboursAboutNewBlockedCell(current)

                solver.solve_constraint(current, 1)

                return None

            else:
                # can only sense neighbours of free cells
                current.confirmed = True
                current.visited = True
                current.total_neighbours = gridObj.cellMap[(current.x, current.y)].total_neighbours
                current.total_blocked_neighbours = gridObj.cellMap[(current.x, current.y)].total_blocked_neighbours

                if alreadyConfirmed == False:
                    current = getKnownNeighbours(current)

                knowledgeGrid.cellMap[(current.x, current.y)] = current
                knowledgeGrid.path.append(current)
                inferenceEngine(current)

                # inform the neighbours that this is free
                if alreadyConfirmed == False:
                    informNeigboursAboutNewFreeCell(current)

                # advanced inference-
                agent_four_inference(current, current.total_blocked_neighbours)
                solver.solve_constraint(current, 0)

    def countBlockedNeighboursInRealGrid():
        # populate the information about blocked neighbours in the real grid
        for i in range(n):
            for j in range(n):
                cell = Cell(i, j, 0, 0, (0, 0))
                cell = senseNeighbours(cell)

                # create object for all cells in the grid
                gridObj.cellMap[(cell.x, cell.y)] = cell

    def repeatedAStar():
        countBlockedNeighboursInRealGrid()

        # mark start as confirmed
        knowledgeGrid.start.confirmed = True
        knowledgeGrid.start.total_blocked_neighbours = gridObj.cellMap[(0, 0)].total_blocked_neighbours
        knowledgeGrid.cellMap[(0, 0)] = knowledgeGrid.start

        print(gridObj.world)
        print()
        agent_four_inference(knowledgeGrid.start, knowledgeGrid.start.total_blocked_neighbours)

        while True:
            if len(knowledgeGrid.path) > 0:
                previousStartPoint = knowledgeGrid.path[len(knowledgeGrid.path) - 1]

                # if we are restarting from the same point where we restarted previously, then dont add this to the path again
                if knowledgeGrid.start.x != previousStartPoint.x and knowledgeGrid.start.y != previousStartPoint.y:
                    knowledgeGrid.path.append(knowledgeGrid.start)

            path = AStar(knowledgeGrid, knowledgeGrid.start)
            if path is None:
                continue

            elif path == False:
                print("Knowledge grid line 279:")
                print(knowledgeGrid.world)
                return False

            else:
                knowledgeGrid.world[0, 0] = 5
                knowledgeGrid.world[knowledgeGrid.start.x, knowledgeGrid.start.y] = 5

                for i in knowledgeGrid.path:
                    print(i.x, i.y)
                    knowledgeGrid.world[i.x, i.y] = 5

                knowledgeGrid.world[knowledgeGrid.goal.x, knowledgeGrid.goal.y] = 5
                print("Knowledge grid line 291:")
                print(knowledgeGrid.world)
                return True

    return repeatedAStar()


PartialSensing(5, 0.3)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 1 1 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]]

1 0
0 1
1 1
Starting from  0 0
1 0
2 0
3 0
3 1
3 2
4 2
4 3
4 4
Visiting 1/0
Cell Iterations :
[[1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
2 0
1 1
2 1
0 1
Solving constraint for  1 0
0 1  is a known free neighbour of  1 0
Solving constraint for  0 1
1 1  is a known free neighbour of  1 0
Solving constraint for  1 1
Visiting 2/0
Cell Iterations :
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
3 0
2 1
3 1
Solving constraint for  2 0
Visiting 3/0
Cell Iterations :
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]]
Blocked by  3 0
Solving constraint for  3 0
Starting from  2 0

Cell location probability function 1 0
cell.total_blocked_neighbours - cell.known_blocked_neighbours): 1 - 0 = 1
cell.total_neighbours:5 -  cell.known_neighbours :2 = 3

Manhattan: 7
Estimate : 9

Cell location probability function 2 0
cell.total_blocked_neighbours - cell.known_blocked_neighbours

True